[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/gee-community/geemap/blob/master/examples/notebooks/55_lidar.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://gishub.org/geemap-binder)

**LiDAR data analysis and visualization with whitebox and geemap**

Create a new conda env to install required packages:

```bash
conda create -n geo python
conda activate geo
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
pip install laspy[lazrs]
```

Uncomment the following line to install packages in Google Colab.

In [ ]:
# !pip install geemap[lidar]

## Import libraries

In [ ]:
import os
import geemap
import whitebox

## Set up whitebox

In [ ]:
wbt = whitebox.WhiteboxTools()
wbt.set_working_dir(os.getcwd())
wbt.set_verbose_mode(False)

## Download sample data

In [ ]:
url = "https://github.com/giswqs/data/raw/main/lidar/madison.laz"
if not os.path.exists("madison.laz"):
    geemap.download_file(url)

## Read LAS/LAZ data

In [ ]:
laz = geemap.read_lidar("madison.laz")

In [ ]:
laz

In [ ]:
str(laz.header.version)

## Upgrade file version

In [ ]:
las = geemap.convert_lidar(laz, file_version="1.4")

In [ ]:
str(las.header.version)

## Write LAS data

In [ ]:
geemap.write_lidar(las, "madison.las")

## Histogram analysis

In [ ]:
wbt.lidar_histogram("madison.las", "histogram.html")

## Visualize LiDAR data

In [ ]:
geemap.view_lidar("madison.las")

## Remove outliers

In [ ]:
wbt.lidar_elevation_slice("madison.las", "madison_rm.las", minz=0, maxz=450)

## Visualize LiDAR data after removing outliers

In [ ]:
geemap.view_lidar("madison_rm.las", cmap="terrain")

## Create DSM

In [ ]:
wbt.lidar_digital_surface_model(
    "madison_rm.las", "dsm.tif", resolution=1.0, minz=0, maxz=450
)

In [ ]:
geemap.add_crs("dsm.tif", epsg=2255)

## Visualize DSM

In [ ]:
m = geemap.Map()
m.add_raster("dsm.tif", colormap="terrain", layer_name="DSM")
m

## Create DEM

In [ ]:
wbt.remove_off_terrain_objects("dsm.tif", "dem.tif", filter=25, slope=15.0)

## Visualize DEM

In [ ]:
m.add_raster("dem.tif", colormap="terrain", layer_name="DEM")
m

## Create CHM

In [ ]:
chm = wbt.subtract("dsm.tif", "dem.tif", "chm.tif")

In [ ]:
m.add_raster("chm.tif", colormap="gist_earth", layer_name="CHM")
m